# Wikipics async demo

[IPython 7.0, Async REPL (from 2018)](https://blog.jupyter.org/ipython-7-0-async-repl-a35ce050f7f7)

In [1]:
from pathlib import Path
from typing import NamedTuple

from IPython.display import display
from ipywidgets.widgets import Image, Layout
import httpx

from wikipics import get_sample_url, get_sample_urls

class ImageRecord(NamedTuple):
    pixels: bytes
    name: str
    size: int

In [ ]:
def fetch_blocking(url) -> ImageRecord:
    resp = httpx.get(url)  # thread blocked here
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))

In [2]:
async def fetch(url) -> ImageRecord:
    async with httpx.AsyncClient() as client:
        resp = await client.get(url)
        resp.raise_for_status()
        name = Path(url).name
        return ImageRecord(resp.content, name, len(resp.content))

In [ ]:
url = get_sample_url(1_000_000)
img_rec = fetch_blocking(url)  # main thread blocks here
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)

In [3]:
url = get_sample_url(1_000_000)
img_rec = await fetch(url)  # console blocks here
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)

   4_458_998 bytes | Mycteria_leucocephala_-_Pak_Thale.jpg


Image(value=b'\xff\xd8\xff\xe1\x13YExif\x00\x00II*\x00\x08\x00\x00\x00\x11\x00\x00\x01\x03\x00\x01\x00\x00\x00…

In [4]:
%%time
img_widgets = []
qty = 10
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0

for url in urls:
    img_rec = await fetch(url)  # SyntaxError: 'await' outside function
    total_bytes += img_rec.size
    display(Image(value=img_rec.pixels, layout=Layout(width='20%')))
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')

print(f'TOTAL BYTES: {total_bytes:_}')

SyntaxError: 'await' outside function (<timed exec>, line 7)

In [ ]:
from gallery import Gallery

In [ ]:
%%timed
num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)

for i, url in enumerate(urls):
    img_rec = fetch(url)
    total_bytes += img_rec.size
    gallery.update(i, img_rec.pixels, url)

print(f'TOTAL BYTES: {gallery.size:_}')

In [5]:
l = 'banana maçã pêra uva'.split()
list(map(str.upper, l))

['BANANA', 'MAÇÃ', 'PÊRA', 'UVA']

In [ ]:
%%time
from concurrent import futures

num_images = 10
gallery = Gallery(num_images)
gallery.display()

urls = get_sample_urls(2_000_000, num_images)
with futures.ThreadPoolExecutor() as pool:
    img_records = pool.map(fetch, urls)
    for i, img_rec in enumerate(img_records):
        gallery.update(i, img_rec.pixels, img_rec.name)

print(f'TOTAL BYTES: {gallery.size:_}')

In [ ]:
%%time
from concurrent import futures

num_images = 10
gallery = Gallery(num_images)
gallery.display()
urls = get_sample_urls(2_000_000, num_images)

with futures.ThreadPoolExecutor() as pool:
    tasks = [pool.submit(fetch, url) for url in urls]
    for i, future in enumerate(futures.as_completed(tasks)):
        img_rec = future.result()
        gallery.update(i, img_rec.pixels, img_rec.name)
        
print(f'TOTAL BYTES: {gallery.size:_}')